# LLM Agent with Tool Calling - Interactive UI

This notebook demonstrates how to create and use an AI agent that:
1. Connects to a local LLM (Qwen3-Coder via LLM Studio)
2. Uses tool calling to execute functions
3. Shows detailed logging of LLM responses and tool execution

## What You'll Learn
- How to structure tool definitions (OpenAI function calling format)
- How LLMs respond with tool calls
- How agents parse and execute these tool calls
- The full request/response cycle with detailed logging

## Setup

Import the agent module and create our agent instance.

In [1]:
# Import the agent
from agent import (
    Agent,
    create_file_tool,
    read_file_tool,
    calculator_tool,
    get_current_time_tool,
    CREATE_FILE_SCHEMA,
    READ_FILE_SCHEMA,
    CALCULATOR_SCHEMA,
    GET_TIME_SCHEMA
)

import json
from IPython.display import display, HTML, Markdown

## Initialize Agent with Tools

Create the agent and register all available tools.

In [2]:
# Create agent instance
# Update the base_url if your LLM Studio is running on a different port
# log_dir: Directory to save request/response JSON files (set to None to disable)
agent = Agent(
    base_url="http://localhost:1234/v1",
    model="qwen/qwen3-coder-30b",
    log_dir="logs"  # All requests/responses will be saved here as JSON files
)

# Register tools
agent.register_tool("create_file", create_file_tool, CREATE_FILE_SCHEMA)
agent.register_tool("read_file", read_file_tool, READ_FILE_SCHEMA)
agent.register_tool("calculator", calculator_tool, CALCULATOR_SCHEMA)
agent.register_tool("get_current_time", get_current_time_tool, GET_TIME_SCHEMA)

print("\n✅ Agent initialized with tools:")
for tool_name in agent.tools.keys():
    print(f"  - {tool_name}")

📁 Logging enabled: /Users/Khaled.Alabsi/projects/llm-agent/logs
✓ Registered tool: create_file
✓ Registered tool: read_file
✓ Registered tool: calculator
✓ Registered tool: get_current_time

✅ Agent initialized with tools:
  - create_file
  - read_file
  - calculator
  - get_current_time


## Understanding Tool Schemas

Let's examine how tools are defined. This is the format the LLM uses to understand what tools are available.

**IMPORTANT**: The complete tool schemas (shown below) are sent to the LLM with EVERY request. The agent now logs the FULL request payload including all tool schemas. Check:
1. Console output - Shows complete request/response
2. `logs/` folder - Saved JSON files for every request/response pair

In [3]:
print("Example Tool Schema (Create File):\n")
print(json.dumps(CREATE_FILE_SCHEMA, indent=2))

print("\n" + "="*80)
print("\nExample Tool Schema (Calculator):\n")
print(json.dumps(CALCULATOR_SCHEMA, indent=2))

Example Tool Schema (Create File):

{
  "type": "function",
  "function": {
    "name": "create_file",
    "description": "Create a new file with the specified content",
    "parameters": {
      "type": "object",
      "properties": {
        "filename": {
          "type": "string",
          "description": "The name of the file to create"
        },
        "content": {
          "type": "string",
          "description": "The content to write to the file"
        }
      },
      "required": [
        "filename",
        "content"
      ]
    }
  }
}


Example Tool Schema (Calculator):

{
  "type": "function",
  "function": {
    "name": "calculator",
    "description": "Perform basic arithmetic operations",
    "parameters": {
      "type": "object",
      "properties": {
        "operation": {
          "type": "string",
          "enum": [
            "add",
            "subtract",
            "multiply",
            "divide"
          ],
          "description": "The arithmetic

## Example 1: File Creation

Watch how the agent:
1. Receives your request
2. Gets the LLM response with tool calls
3. Executes the `create_file` tool
4. Returns the final result

In [4]:
# Reset conversation for a clean start
agent.reset_conversation()

# Run the agent
response = agent.run(
    "Create a file called 'hello.txt' with the content 'Hello from the AI agent!'"
)

print("\n" + "="*80)
print("📝 FINAL AGENT RESPONSE:")
print("="*80)
print(response)

🔄 Conversation history reset

🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 
AGENT RUN STARTED: 2025-11-09 14:35:38
🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 

────────────────────────────────────────────────────────────────────────────────
ITERATION 1/5
────────────────────────────────────────────────────────────────────────────────

📤 SENDING TO LLM - COMPLETE REQUEST PAYLOAD
{
  "model": "qwen/qwen3-coder-30b",
  "messages": [
    {
      "role": "user",
      "content": "Create a file called 'hello.txt' with the content 'Hello from the AI agent!'"
    }
  ],
  "temperature": 0.7,
  "max_tokens": 2000,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "create_file",
        "description": "Create a new file with the specified content",
        "parameters": {
          "type": "object",
          "properties": {
            "filename": {
              "type": "string",
            

## Example 2: File Read + Create Chain

This demonstrates multi-step tool usage where the agent might need to:
1. Read a file
2. Process the content
3. Create a new file

In [ ]:
# Reset conversation
agent.reset_conversation()

# Run the agent
response = agent.run(
    "Read the file 'hello.txt' and create a new file called 'hello_copy.txt' with the same content but in uppercase"
)

print("\n" + "="*80)
print("📝 FINAL AGENT RESPONSE:")
print("="*80)
print(response)

## Example 3: Calculator Tool

See how the agent handles mathematical operations using the calculator tool.

In [ ]:
# Reset conversation
agent.reset_conversation()

# Run the agent
response = agent.run(
    "Calculate 127 multiplied by 89, then divide the result by 3"
)

print("\n" + "="*80)
print("📝 FINAL AGENT RESPONSE:")
print("="*80)
print(response)

## Example 4: Time Tool

Simple example with no parameters.

In [ ]:
# Reset conversation
agent.reset_conversation()

# Run the agent
response = agent.run(
    "What's the current time?"
)

print("\n" + "="*80)
print("📝 FINAL AGENT RESPONSE:")
print("="*80)
print(response)

## Example 5: Complex Multi-Tool Task

Give the agent a complex task that requires multiple tools.

In [ ]:
# Reset conversation
agent.reset_conversation()

# Run the agent with a complex task
response = agent.run(
    """Create a report file called 'calculation_report.txt' that contains:
    1. The current date and time
    2. The result of 456 + 789
    3. The result of 100 multiplied by 25
    Format it nicely with headers.
    """
)

print("\n" + "="*80)
print("📝 FINAL AGENT RESPONSE:")
print("="*80)
print(response)

## Interactive: Try Your Own Prompt

Enter your own prompt to see how the agent handles it!

In [ ]:
# Reset conversation
agent.reset_conversation()

# Your custom prompt here
user_prompt = "Calculate 50 divided by 2, then create a file called 'result.txt' with the answer"

response = agent.run(user_prompt)

print("\n" + "="*80)
print("📝 FINAL AGENT RESPONSE:")
print("="*80)
print(response)

## Inspect Conversation History

See the full conversation history including tool calls and results.

In [ ]:
print("Conversation History:\n")
print(json.dumps(agent.conversation_history, indent=2))

In [ ]:
# Load and display the most recent request file
request_files = sorted(glob.glob("logs/request_*.json"))
if request_files:
    with open(request_files[-1], 'r') as f:
        request_data = json.load(f)
    
    print("="*80)
    print("COMPLETE REQUEST PAYLOAD SENT TO LLM")
    print("="*80)
    print(json.dumps(request_data, indent=2))
else:
    print("No request files found yet. Run an agent task first!")

### Examine a Request File

Let's open one of the request files to see the COMPLETE payload sent to the LLM, including all tool schemas:

In [ ]:
import os
import glob

# List all log files
log_files = sorted(glob.glob("logs/*.json"))

print(f"Total log files: {len(log_files)}\n")
print("Recent files:")
for f in log_files[-10:]:  # Show last 10 files
    size = os.path.getsize(f)
    print(f"  {f} ({size:,} bytes)")

## View Saved Log Files

All requests and responses are saved to the `logs/` directory. Let's see what's been saved:

## Understanding the Response Flow

Here's what happens step-by-step:

1. **User Message** → Agent receives your request
2. **LLM Call** → Agent sends message + tool schemas to LLM
3. **LLM Response** → LLM returns either:
   - Text response (no tools needed)
   - Tool call(s) in this format:
   ```json
   {
     "tool_calls": [
       {
         "id": "call_abc123",
         "type": "function",
         "function": {
           "name": "create_file",
           "arguments": "{\"filename\": \"test.txt\", \"content\": \"Hello\"}"
         }
       }
     ]
   }
   ```
4. **Tool Execution** → Agent executes each tool call
5. **Tool Results** → Results are sent back to LLM
6. **Final Response** → LLM generates final answer based on tool results

The detailed logs show each step!

## Create Your Own Tool

Here's how to create and register a custom tool:

In [ ]:
# Define a custom tool function
def word_count_tool(text: str) -> dict:
    """Count words in text"""
    words = text.split()
    return {
        "status": "success",
        "text": text,
        "word_count": len(words),
        "character_count": len(text)
    }

# Define the schema
WORD_COUNT_SCHEMA = {
    "type": "function",
    "function": {
        "name": "word_count",
        "description": "Count the number of words and characters in a text",
        "parameters": {
            "type": "object",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "The text to analyze"
                }
            },
            "required": ["text"]
        }
    }
}

# Register the tool
agent.register_tool("word_count", word_count_tool, WORD_COUNT_SCHEMA)

# Test it
agent.reset_conversation()
response = agent.run("Count the words in this sentence: 'The quick brown fox jumps over the lazy dog'")

print("\n" + "="*80)
print("📝 FINAL AGENT RESPONSE:")
print("="*80)
print(response)

## Next Steps

Try experimenting with:
1. Creating more complex tools (API calls, data processing, etc.)
2. Different agent frameworks (LangChain, AutoGen, CrewAI)
3. Adding memory to the agent
4. Building multi-agent systems
5. Testing different LLM models

The key concepts you've learned:
- Tool definition structure (OpenAI function calling format)
- How LLMs return tool calls
- How to parse and execute tool calls
- The full agent loop with multiple iterations